# 1. Read data and explore

In [378]:
import pandas as pd

# assuming data files in the same folder as this notebook
train_data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [379]:
train_data.head()
# read data_description.txt for details

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [380]:
train_data.corr()["SalePrice"].sort_values(ascending = False)[:8]

SalePrice      1.000000
OverallQual    0.790982
GrLivArea      0.708624
GarageCars     0.640409
GarageArea     0.623431
TotalBsmtSF    0.613581
1stFlrSF       0.605852
FullBath       0.560664
Name: SalePrice, dtype: float64

*The following features have the highest correlation to sales prices: Overall quality, above ground living area size, garage size, basement size, number of bathrooms etc.*

# 2. Data preparation

In general the following preparation steps are needed:

* Fill in missing values
    * For categorical features, take the most frequent value e.g. MSZoning
    * Add missing category if the none category was interpreted as NA to pandas e.g. BsmtQual
    * For numerical features, take the median value e.g. LotFrontage
* Scale the numerical features e.g. LotArea
* Discretize year features into one hot category e.g. YearBuilt
* One-hot encode the categorical values that has no order e.g. MSSubClass
* Ordinal encode the categorical values that has order e.g. OverallQual

In [381]:
# merge train and test to find missing values
merged = pd.concat([train, test])
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2627 entries, 254 to 1458
Data columns (total 81 columns):
1stFlrSF         2627 non-null int64
2ndFlrSF         2627 non-null int64
3SsnPorch        2627 non-null int64
Alley            181 non-null object
BedroomAbvGr     2627 non-null int64
BldgType         2627 non-null object
BsmtCond         2554 non-null object
BsmtExposure     2555 non-null object
BsmtFinSF1       2626 non-null float64
BsmtFinSF2       2626 non-null float64
BsmtFinType1     2557 non-null object
BsmtFinType2     2557 non-null object
BsmtFullBath     2625 non-null float64
BsmtHalfBath     2625 non-null float64
BsmtQual         2555 non-null object
BsmtUnfSF        2626 non-null float64
CentralAir       2627 non-null object
Condition1       2627 non-null object
Condition2       2627 non-null object
Electrical       2626 non-null object
EnclosedPorch    2627 non-null int64
ExterCond        2627 non-null object
ExterQual        2627 non-null object
Exterior1st      2

In [382]:
# Fill in the missing values
def fill_missing(original_df):
    
    df = original_df.copy()
    
    # numerical
    df["BsmtFinSF1"].fillna(np.mean(train_data["BsmtFinSF1"]), inplace = True)
    df["BsmtFinSF2"].fillna(np.mean(train_data["BsmtFinSF2"]), inplace = True)
    df["BsmtFullBath"].fillna(np.mean(train_data["BsmtFullBath"]), inplace = True)
    df["BsmtHalfBath"].fillna(np.mean(train_data["BsmtHalfBath"]), inplace = True)
    df["BsmtUnfSF"].fillna(np.mean(train_data["BsmtUnfSF"]), inplace = True)
    df["GarageArea"].fillna(np.mean(train_data["GarageArea"]), inplace = True)
    df["GarageCars"].fillna(np.mean(train_data["GarageCars"]), inplace = True)
    df["GarageYrBlt"].fillna(np.mean(train_data["GarageYrBlt"]), inplace = True)
    df["LotFrontage"].fillna(np.mean(train_data["LotFrontage"]), inplace = True)
    df["MasVnrArea"].fillna(np.mean(train_data["MasVnrArea"]), inplace = True)
    df["TotalBsmtSF"].fillna(np.mean(train_data["TotalBsmtSF"]), inplace = True)
    
    # categorical
    df["BsmtCond"].fillna(train_data["BsmtCond"].value_counts().index[0], inplace = True)
    df["BsmtExposure"].fillna(train_data["BsmtExposure"].value_counts().index[0], inplace = True)
    df["BsmtFinType1"].fillna(train_data["BsmtFinType1"].value_counts().index[0], inplace = True)
    df["BsmtFinType2"].fillna(train_data["BsmtFinType2"].value_counts().index[0], inplace = True)
    df["BsmtQual"].fillna(train_data["BsmtQual"].value_counts().index[0], inplace = True)
    df["Electrical"].fillna(train_data["Electrical"].value_counts().index[0], inplace = True)
    df["Exterior1st"].fillna(train_data["Exterior1st"].value_counts().index[0], inplace = True)
    df["Exterior2nd"].fillna(train_data["Exterior2nd"].value_counts().index[0], inplace = True)
    df["Functional"].fillna(train_data["Functional"].value_counts().index[0], inplace = True)
    df["GarageCond"].fillna(train_data["GarageCond"].value_counts().index[0], inplace = True)
    df["GarageFinish"].fillna(train_data["GarageFinish"].value_counts().index[0], inplace = True)
    df["GarageQual"].fillna(train_data["GarageQual"].value_counts().index[0], inplace = True)
    df["GarageType"].fillna(train_data["GarageType"].value_counts().index[0], inplace = True)
    df["KitchenQual"].fillna(train_data["KitchenQual"].value_counts().index[0], inplace = True)
    df["MSSubClass"].fillna(train_data["MSSubClass"].value_counts().index[0], inplace = True)
    df["MSZoning"].fillna(train_data["MSZoning"].value_counts().index[0], inplace = True)
    df["MasVnrType"].fillna(train_data["MasVnrType"].value_counts().index[0], inplace = True)
    df["SaleType"].fillna(train_data["SaleType"].value_counts().index[0], inplace = True)
    df["Utilities"].fillna(train_data["Utilities"].value_counts().index[0], inplace = True)

    # Fill NA's with explicit categorical value
    df["Alley"].fillna("NoAlley", inplace = True)
    df["Fence"].fillna("NoFence", inplace = True)
    df["FireplaceQu"].fillna("NoFireplace", inplace = True)
    df["MiscFeature"].fillna("NoFeature", inplace = True)
    df["PoolQC"].fillna("NoPool", inplace = True)
    
    return df

In [383]:
# Seperate features and label
train_x = train_data.drop(["SalePrice", "Id"], axis = 1)
train_y = train_data["SalePrice"].copy()

test_x = test.drop(["Id"], axis = 1)

In [384]:
# fill missing
train_filled = fill_missing(train_x)
test_filled = fill_missing(test_x)

merged_filled = pd.concat([train_filled, test_filled])

In [385]:
# Find out categorical features with differing number of values in train and test set
ordinals = ["OverallQual", "OverallCond", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageQual", "GarageCond", "PoolQC", "Fence"]
onehots = ["MSSubClass", "MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating", "CentralAir", "Electrical", "GarageType", "GarageFinish", "PavedDrive", "MiscFeature", "MoSold", "SaleType", "SaleCondition"]

all_categories = ordinals + onehots

for cat in all_categories:
    train_values = set(train_filled[cat].unique())
    test_values = set(test_filled[cat].unique())
    if train_values - test_values:
        print(f"Categorical feature: {cat} has differing number of values in train {train_values} and test {test_values}")

Categorical feature: GarageQual has differing number of values in train {'Fa', 'Ex', 'Gd', 'Po', 'TA'} and test {'Gd', 'Po', 'Fa', 'TA'}
Categorical feature: PoolQC has differing number of values in train {'Gd', 'NoPool', 'Ex', 'Fa'} and test {'Gd', 'NoPool', 'Ex'}
Categorical feature: Utilities has differing number of values in train {'AllPub', 'NoSeWa'} and test {'AllPub'}
Categorical feature: Condition2 has differing number of values in train {'Feedr', 'PosN', 'Artery', 'RRNn', 'PosA', 'RRAn', 'Norm', 'RRAe'} and test {'Feedr', 'PosN', 'Artery', 'PosA', 'Norm'}
Categorical feature: HouseStyle has differing number of values in train {'2Story', 'SLvl', '2.5Unf', '1.5Fin', 'SFoyer', '1Story', '2.5Fin', '1.5Unf'} and test {'2Story', 'SLvl', '2.5Unf', '1.5Fin', 'SFoyer', '1Story', '1.5Unf'}
Categorical feature: RoofMatl has differing number of values in train {'WdShngl', 'Membran', 'Roll', 'CompShg', 'ClyTile', 'WdShake', 'Tar&Grv', 'Metal'} and test {'WdShake', 'Tar&Grv', 'WdShngl', 'Co

In [386]:
# Full pipe with column transformer on each feature that needs data prep
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer

full_pipe = ColumnTransformer([
    # special treatment for encoding categorical features that have differing number of values in train and test set
    ("MSSubClass", OneHotEncoder(categories = [sorted(merged_filled["MSSubClass"].unique())]), ["MSSubClass"]), 
    ("Utilities", OneHotEncoder(categories = [sorted(merged_filled["Utilities"].unique(), key = lambda x : float('-inf') if x is np.nan else x)]), ["Utilities"]), 
    ("Condition2", OneHotEncoder(categories = [sorted(merged_filled["Condition2"].unique())]), ["Condition2"]), 
    ("HouseStyle", OneHotEncoder(categories = [sorted(merged_filled["HouseStyle"].unique())]), ["HouseStyle"]), 
    ("RoofMatl", OneHotEncoder(categories = [sorted(merged_filled["RoofMatl"].unique())]), ["RoofMatl"]), 
    ("Exterior1st", OneHotEncoder(categories = [sorted(merged_filled["Exterior1st"].unique())]), ["Exterior1st"]), 
    ("Exterior2nd", OneHotEncoder(categories = [sorted(merged_filled["Exterior2nd"].unique())]), ["Exterior2nd"]), 
    ("Heating", OneHotEncoder(categories = [sorted(merged_filled["Heating"].unique())]), ["Heating"]), 
    ("Electrical", OneHotEncoder(categories = [sorted(merged_filled["Electrical"].unique())]), ["Electrical"]), 
    ("MiscFeature", OneHotEncoder(categories = [sorted(merged_filled["MiscFeature"].unique())]), ["MiscFeature"]), 
    
    ("GarageQual", OrdinalEncoder(categories = [sorted(merged_filled["GarageQual"].unique())]), ["GarageQual"]), 
    ("PoolQC", OrdinalEncoder(categories = [sorted(merged_filled["PoolQC"].unique())]), ["PoolQC"]), 
    
    ("discretize_year", KBinsDiscretizer(n_bins=10, encode='onehot'), ["GarageYrBlt", "YearBuilt", "YearRemodAdd", "YrSold"]),
    ("std_scaling", StandardScaler(), ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal"]),
    ("onehot_encode", OneHotEncoder(), ["MSZoning", "Street", "Alley", "LotShape", "LandContour", "LotConfig", "LandSlope", "Neighborhood", "Condition1", "BldgType", "RoofStyle", "MasVnrType", "Foundation", "CentralAir", "GarageType", "GarageFinish", "PavedDrive", "MoSold", "SaleType", "SaleCondition"]),
    ("ordinal_encode", OrdinalEncoder(), ["OverallQual", "OverallCond", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageCond", "Fence"]),
], remainder = "passthrough")

In [387]:
# run data prep pipeline on all data segments
train_prepared = full_pipe.fit_transform(train_filled)
test_prepared = full_pipe.fit_transform(test_filled)

In [388]:
train_prepared.shape

(1460, 289)

In [389]:
test_prepared.shape

(1459, 289)

# 3. Model selection

In [390]:
import sklearn.linear_model
import sklearn.tree
import sklearn.ensemble
import sklearn.kernel_ridge
import sklearn.neighbors
import sklearn.svm
import sklearn.cross_decomposition
import sklearn.compose

from sklearn.metrics import mean_squared_error
import numpy as np

import warnings

class Regressor:
    def __init__(self, label, model, score = float("inf")):
        self.label = label
        self.model = model
        self.score = score

def fit_eval(regressor):
    warnings.filterwarnings('ignore')
    try:
        regressor.model.fit(train_prepared.toarray(), train_y)
        predictions = regressor.model.predict(train_prepared.toarray())
        mse = mean_squared_error(train_y, predictions)
        regressor.score = np.sqrt(mse)
    except ValueError:
        print(f"ValueError while training {regressor.label}")

In [391]:
import sklearn.utils.testing
from collections import namedtuple

# train and evaluate all regressors available in sklearn
regressors = []
for r in sklearn.utils.testing.all_estimators(type_filter='regressor'):
    regressor = Regressor(r[0], r[1]())
    fit_eval(regressor)
    print(f"{regressor.label} MSE: {regressor.score}")
    regressors.append(regressor)

ARDRegression MSE: 28263.525035700437
AdaBoostRegressor MSE: 28089.877661936098
BaggingRegressor MSE: 16916.652784617938
BayesianRidge MSE: 26472.339294274174
CCA MSE: 67071.86911582811
DecisionTreeRegressor MSE: 0.0
ElasticNet MSE: 31851.88586933182
ElasticNetCV MSE: 73475.69246408706
ExtraTreeRegressor MSE: 0.0
ExtraTreesRegressor MSE: 0.0
GaussianProcessRegressor MSE: 1.9670129597729957e-05
GradientBoostingRegressor MSE: 14867.920124221744
HuberRegressor MSE: 29387.241292964638
KNeighborsRegressor MSE: 31525.988512073178
KernelRidge MSE: 23321.801490892973
Lars MSE: 1.3104456923887348e+16
LarsCV MSE: 1.3104575718962656e+16
Lasso MSE: 21846.373271707245
LassoCV MSE: 24264.063878395704
LassoLars MSE: 23338.850585343138
LassoLarsCV MSE: 24480.18966651223
LassoLarsIC MSE: 28218.579759283723
LinearRegression MSE: 21844.280905681044
LinearSVR MSE: 82815.41760949198
MLPRegressor MSE: 170798.3984823012
ValueError while training MultiTaskElasticNet
MultiTaskElasticNet MSE: inf
ValueError whi

*From all regressors available in sklearn - BaggingRegressor, GradientBoostingRegressor and RandomForestRegressor have lowest mean squared root error. The regressors are all used with default hyper parameters. Some regressors produced RMSE of zero, which is not plausible and therefore omitted from choices of model.*

# 4. Hyper parameter tuning

From best performing models the GradientBoostingRegressor was chosen. Now let us try to tune the hyperparameters using RandomizedSearchCV.

In [392]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Search space for hyper parameters
hyperparameters = {'n_estimators':[500,1000,2000],
                   'learning_rate':[.001,0.01,.1],
                   'max_depth':[1,2,4],
                   'subsample':[.5,.75,1],
                   'random_state':[1]}

random_cv = RandomizedSearchCV(estimator=GradientBoostingRegressor(),
            param_distributions=hyperparameters,
            cv=5, n_iter=100,
            scoring = 'neg_mean_squared_error',n_jobs = -1,
            verbose = 5, 
            return_train_score = True,
            random_state=42)
random_cv.fit(train_prepared, train_y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 21.7min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [500, 1000, 2000], 'learning_rate': [0.001, 0.01, 0.1], 'max_depth': [1, 2, 4], 'subsample': [0.5, 0.75, 1], 'random_state': [1]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=5)

In [394]:
selected_model = random_cv.best_estimator_
selected_model.fit(train_prepared.toarray(), train_y)
predictions = selected_model.predict(train_prepared.toarray())
mse = mean_squared_error(train_y, predictions)
print(np.sqrt(mse))

5606.726800507456


# 5. Predictions

In [398]:
test_predictions = selected_model.predict(test_prepared)
submission = pd.DataFrame({"Id": test["Id"], "SalePrice": test_predictions})
submission.to_csv("submission.csv", index = False)
# This submission resulted in Root Mean Square Logarathmic Error of 0.1415